# **Churn Server**


in the following section we create a new model serving function which wraps our class , and specify model and other resources.
Deploying the serving function will provide us an http endpoint that can handle requests in real time.
This function is part of the [customer-churn-prediction demo](https://github.com/mlrun/demos/tree/master/customer-churn-prediction).<br>
To see how the model is trained or how the data-set is generated, check out `coxph_trainer` and `xgb_trainer` functions from the function marketplace repository.

### **Steps**
1. [Setup function parameters](#Setup-function-parameters)
2. [Importing the function](#Importing-the-function)
3. [Testing the function locally](#Testing-the-function-locally)
4. [Testing the function remotely](#Testing-the-function-remotely)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Following packages are required, make sure to install
# !pip install xgboost==1.3.1

### **Setup function parameters**

In [3]:
# Setting up models path
xgb_model_path = 'https://s3.wasabisys.com/iguazio/models/function-marketplace-models/churn_server/xgb_model.pkl'

### **Importing the function**

In [4]:
import mlrun
mlrun.set_environment(project='function-marketplace')

# Importing the function from the hub
fn = mlrun.import_function("hub://churn_server:development")
fn.apply(mlrun.auto_mount())

# Manually specifying needed packages 
fn.spec.build.commands = ['pip install lifelines==0.22.8', 'pip install xgboost==1.3.1']

# Adding the model 
fn.add_model(key='xgb_model', model_path=xgb_model_path ,class_name='ChurnModel')

> 2021-10-14 06:10:16,104 [info] loaded project function-marketplace from MLRun DB


### **Testing the function locally**

> Note that this function is a serving function, hence not needs to run, but deployed.<br>

in order to test locally without deploying to server, mlrun provides mocking api that simulate the action.

In [5]:
# When mocking, class has to be present
from churn_server import *

# Mocking function
server = fn.to_mock_server()

> 2021-10-14 06:10:19,145 [info] model xgb_model was loaded
> 2021-10-14 06:10:19,145 [info] Initializing endpoint records
> 2021-10-14 06:10:19,164 [info] Loaded ['xgb_model']


In [6]:
import pandas as pd

#declaring test_set path
test_set_path = "https://s3.wasabisys.com/iguazio/data/function-marketplace-data/churn_server/test_set.csv"

# Getting the data
x_test = pd.read_csv(test_set_path)
y_test = x_test['labels']
x_test.drop(['labels'],axis=1,inplace=True)
x_test.head()

,gender,senior,partner,deps,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,PaperlessBilling,MonthlyCharges,tenure_map,ISP_1,ISP_2,Contract_1,Contract_2,Payment_1,Payment_2,Payment_3
0,0,0,1,0,27,1,0,1,0,0,...,1,101.90,2.0,1,0,1,0,1,0,0
1,0,1,0,0,1,1,1,0,0,0,...,1,85.70,0.0,1,0,0,0,0,1,0
2,1,0,0,0,1,1,0,0,0,0,...,1,69.55,0.0,1,0,0,0,0,1,0
3,0,0,0,0,53,1,1,0,1,1,...,0,105.55,4.0,1,0,0,1,0,1,0
4,0,0,0,0,43,1,1,0,1,1,...,1,104.60,3.0,1,0,0,1,0,1,0


In [12]:
# KFServing protocol event
event_data = {"inputs": x_test.values.tolist()}

In [13]:
response = server.test(path='/v2/models/xgb_model/predict',body=event_data)

In [14]:
print(f'When mocking to server, returned dict has the following fields : {", ".join([x for x in response.keys()])}')

When mocking to server, returned dict has the following fields : id, model_name, outputs


### **Testing the function remotely**

In [10]:
address = fn.deploy()

> 2021-10-14 06:10:20,163 [info] Starting remote function deploy
2021-10-14 06:10:20  (info) Deploying function
2021-10-14 06:10:20  (info) Building
2021-10-14 06:10:20  (info) Staging files and preparing base images
2021-10-14 06:10:20  (info) Building processor image
2021-10-14 06:10:21  (info) Build complete
2021-10-14 06:10:29  (info) Function deploy complete
> 2021-10-14 06:10:30,408 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-function-marketplace-churn-server.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.dev39.lab.iguazeng.com:31984']}


In [11]:
import json
import requests

# using requests to predict
response = requests.put(address + "/v2/models/xgb_model/predict", json=json.dumps(event_data))

# returned data is a string 
y_predict = json.loads(response.text)['outputs']
accuracy = sum(1 for x,y in zip(y_predict,y_test) if x == y) / len(y_test)
print(f"model's accuracy : {accuracy}")

model's accuracy : 0.7913907284768212


[Back to the top](#Churn-Server)